# Imports

In [ ]:
import pysweep
import pysweep.databackends.debug
import pysweep.databackends.list_backend
import pysweep.databackends.qcodes
import pysweep.core.measurementfunctions
from pysweep.core.sweepobject import SweepObject
from pysweep.core.measurementfunctions import MakeMeasurementFunction
from pysweep.databackends.base import DataParameterFixedAxis
from pysweep.databackends.base import DataParameter
import pysweep.convenience as conv

import qcodes as qc
from qcodes.dataset.measurements import Measurement
import qcodes.dataset.plotting

from cqed.utils.datahandling import create_local_dbase_in
import cqed.custom_pysweep_functions.vna as cvna
import cqed.custom_pysweep_functions.magnet as cmgnt

import numpy as np
import matplotlib.pyplot as plt

# Set up station and QCoDeS database

## Set up measurement station

### K1

In [ ]:
station = qc.Station(config_file=r'D:\OneDrive\Setups\LK1\LK1_code\code\cqed\station_init\LK1.station.yaml')

### K2

In [ ]:
station = qc.Station(config_file=r'D:\OneDrive\Setups\LK2\LK2_code\code\cqed\station_init\LK2.station.yaml')

## Set up instruments

In [ ]:
vna = station.load_instrument('vna')

vna.add_channel('S21')
station.vna.S21.power(-50)
station.vna.S21.start(4e9)
station.vna.S21.stop(8.5e9)
station.vna.S21.bandwidth(1e3)
station.vna.S21.npts(10001)

# Automatically remove the cable delay from the phase signal
# Should be checked in large frequency range and with higer readout power
# Often can be fine-tuned a bit better by hand.
station.vna.S21.set_electrical_delay_auto()

In [ ]:
mgnt = station.load_instrument('mgnt', field_limits=cmgnt.field_limit)
print('current X-field:', station.mgnt.x_measured(), 'T')
print('current Y-field:',station.mgnt.y_measured(), 'T')
print('current Z-field:',station.mgnt.z_measured(), 'T')

In [ ]:
ivvi = station.load_ivvi()
ivvi.print_readable_snapshot()

## Pass measurement station to pysweep 


### To be removed after merging of pysweep pull request #19

In [ ]:
pysweep.STATION = station

## Name and initialize database

In [ ]:
# To place the database in D:/Data/MyExperiment/ use:
folder = 'MyExperiment'

# To place the database in a more nested folder structure, e.g. D:/Data/MyExperiment/TestSample/FirstMeasurements/, use:
# folder = 'MyExperiment/TestSample/FirstMeasurements'

# Name your database
database_name = 'yyyy_mm_dd_Run1'

create_local_dbase_in(folder_name=folder, db_name='%s.db' %(database_name))

qc.config['core']['db_location']

# Define pysweep functions

In [ ]:
# Function called before beginning of measurement loop.
# This is a good place to define the VNA sweep parameters. Can also be made a function with changable parameters.

def init_measurement(d):
    # some exemplary parameters that could be set in the init function
    station.vna.rf_on()
    station.vna.S21.start(4e9)
    station.vna.S21.stop(5e9)
    station.vna.S21.npts(1001)
    
    station.vna.S21.power(-20)
    station.vna.S21.bandwidth(1e3)
    station.vna.S21.avg(10)
    
    print('Starting measurement.')
    
# Function called after the end of the measurement.
def end_measurement(d):
    print('Measurement finished.')

# Single VNA trace

This just records a single VNA trace with the VNA parameters currently set or set (if specified) in the init_measurement function.

In [ ]:
# Give your sample a meaningful name.
sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='single_VNA_trace', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cvna.return_vna_trace,
                 databackend = pysweep.databackends.qcodes.DataBackend(meas))

For plotting of the generated dataset in the notebook directly

In [ ]:
qc.dataset.plotting.plot_dataset(result.datasaver.dataset)

## Measuring multiple resonators simultaneously

Measure multiple resonators at different frequencies with a span specified for each resonator. Useful for example when ramping up magnetic field and measuring multiple resonators at the same time to save measurement time. 

Below is an example for a powersweep of 4 resonators simulatenously. Which means, the power parameter is swept, all four resonators with their corresponding span are measured, then the next power is measured and so on.

In [ ]:
# Set centerfrequencies for individual resonators
fr = [5.0e9, 5.5e9, 6.0e9, 6.5e9]
# Set measurement spans for individual resonators
spans = [20e6, 20e6, 20e6, 20e6]

sample_name = 'sample1_all_resonators'
experiment_name = 'VNA_vs_pwr'

exp = qc.load_or_create_experiment(experiment_name=experiment_name, sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, 
                       eval(cvna.multiple_meas_functions(fr, spans)), 
                       pysweep.sweep_object(station.vna.S21.power, np.arange(-50, -15, 5)),
                       databackend = pysweep.databackends.qcodes.DataBackend(meas))

# Power sweep

In [ ]:
# Give your sample a meaningful name.
sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='VNA_vs_pwr', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cvna.return_vna_trace, 
                  pysweep.sweep_object(station.vna.S21.power, np.arange(-50, -15, 5)),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))

# Gate sweep

Exemplary setup to measure a spectrum with the VNA (as specified in the init_measurement function for example) as a function of the dac1 voltage of the IVVI

In [ ]:
# Give your sample a meaningful name.
sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='VNA_vs_gate', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cvna.return_vna_trace, 
                  pysweep.sweep_object(station.ivvi.dac1, np.arange(0, 1010, 10)),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))

# Magnetic field sweep

## Setting of x, y, z magnetic fields

In [ ]:
station.mgnt.field_ramp_rate(FieldVector(x=3e-3, y=3e-3, z=3e-3))

In [ ]:
# Get current field strength of x, y, and z component.
print(station.mgnt.x_measured())
print(station.mgnt.y_measured())
print(station.mgnt.z_measured())

In [ ]:
# Set all target field components to some (reasonable) value and ramp magnetic field.
# After ramping, check field strength
station.mgnt.x_target(0.0e-3)
station.mgnt.y_target(0.0e-3)
station.mgnt.z_target(0.1e-3)

station.mgnt.ramp(mode='safe')

print(station.mgnt.r_measured())
print(station.mgnt.phi_measured())
print(station.mgnt.theta_measured())

## Sweeping in spherical coordinates

The sweep functions below explicitly calculate x, y, and z components from the spherical component input. This is done for greater control of the actual sweeping direction of the magnet

For the actual measurement, we make use of the '+' functionality of pysweep, to record the measured magnetic field components along with the desired setpoints.

### Phi sweep

In [ ]:
# Give your sample a meaningful name.
sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='VNA_vs_mgnt-phi', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cmgnt.measure_magnet_components + cvna.return_vna_trace, 
                  cmgnt.sweep_phi(r = 3.0e-3, theta = 90, points = np.arange(0, 360, 5)),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))

### Theta sweep

In [ ]:
# Give your sample a meaningful name.
sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='VNA_vs_mgnt-theta', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cmgnt.measure_magnet_orientation + cvna.return_vna_trace, 
                  cmgnt.sweep_theta(r = 3.0e-3, phi = 90, points = np.arange(0, 180, 5)),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))

### r sweep

In [ ]:
# Give your sample a meaningful name.
sample_name = 'sample1'

exp = qc.load_or_create_experiment(experiment_name='VNA_vs_mgnt-r', sample_name=sample_name)
meas = Measurement(exp, station)

result = pysweep.sweep(init_measurement, end_measurement, cmgnt.measure_magnet_orientation + cvna.return_vna_trace, 
                  cmgnt.sweep_r(phi = 90, theta = 0, points = np.arange(0, 20e-3, 2e-3)),
                  databackend = pysweep.databackends.qcodes.DataBackend(meas))